In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import plotly.graph_objects as go
from scipy.stats import poisson

In [ ]:
all_dmv = pd.read_csv('datasets/vehicle_fuel_type.csv')
all_dmv.columns = all_dmv.columns.str.replace(" ", "_")
all_dmv.head()

In [ ]:
all_dmv['Date'].value_counts()

In [ ]:
sdge_zip_csv = pd.read_csv('data/SDGE_zip.csv')
sdge_service_zip = sdge_zip_csv['ZipCode']

In [ ]:
all_dmv['Zip_Code'] = all_dmv['Zip_Code'].astype(str).str.strip()
sdge_service_zip = sdge_service_zip.astype(str).str.strip()

In [ ]:
SDGE_dmv = all_dmv[(all_dmv['Zip_Code'].isin(sdge_service_zip)) & (all_dmv['Fuel'] == 'Battery Electric')]
SDGE_dmv

In [ ]:
# Function to set a Year column for each date to organize DMV data
def map_date_to_year(date):
    if date == "12/31/2022":
        return 2022
    elif date == "1/1/2022":
        return 2021
    elif date == "1/1/2021":
        return 2020
    elif date == "1/1/2020":
        return 2019
    elif date == "10/1/2018":
        return 2018
    elif date == "12/31/2023":
        return 2023
    else:
        return None  # Handle unexpected or missing values

In [ ]:
# Apply the function to the 'Date' column to create the new 'Year' column
SDGE_dmv['Year'] = SDGE_dmv['Date'].apply(map_date_to_year)

In [ ]:
SDGE_dmv['Date'].value_counts()

In [ ]:
SDGE_dmv['Year'].value_counts()

In [ ]:
SDGE_dmv.head()

## Top 20 ZIP Codes with Highest Number Of EVs

In [ ]:
vehicles_by_zip = SDGE_dmv.groupby('Zip_Code')['Vehicles'].sum().sort_values(ascending=False).reset_index()
vehicles_by_zip.columns = ['zip', 'ev_owners']
vehicles_by_zip

In [ ]:
# Create a bar plot for the top 20 ZIP Codes with the highest number of vehicles
plt.figure(figsize=(14, 7))
top_20 = vehicles_by_zip.head(20)
plt.bar(top_20['zip'], top_20['ev_owners'], color='skyblue')
plt.title('Top 20 ZIP Codes with the Highest Number of EVs')
plt.xlabel('ZIP Code')
plt.ylabel('Number of Vehicles')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Total EV Count Per Year From 2018 - 2023

In [ ]:
# Group data by YEAR and sum the number of EVs
ev_distribution_by_year = SDGE_dmv.groupby('Year')['Vehicles'].sum().sort_index()
ev_distribution_by_year

In [ ]:
# Create a bar plot for the distribution of EVs throughout the years
plt.figure(figsize=(12, 6))
ev_distribution_by_year.plot(kind='bar', color='green')
plt.title('Growth of EVs Throughout the Years (Bar)')
plt.xlabel('Year')
plt.ylabel('Number of EVs')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize = (12, 6))
ev_distribution_by_year.plot(kind='line', color='green')
plt.title('Growth of EVs Throughout the Years (Line)')
plt.xlabel('Year')
plt.ylabel('Number of EVs')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Count of New EV Registered Per Year From 2018 - 2023

In [ ]:
yearly_growth = ev_distribution_by_year.diff()
growth_labels = [f"{year1} - {year2}" for year1, year2 in zip(ev_distribution_by_year.index[:-1], ev_distribution_by_year.index[1:])]

yearly_growth_df = pd.DataFrame(
    {"Label" : growth_labels,
    "Yearly Growth": yearly_growth[1:]}
)
yearly_growth_df.set_index("Label")

In [ ]:
# Bar Graph for Yearly Growth with Custom Labels
plt.figure(figsize=(10, 6))
plt.bar(yearly_growth_df["Label"], yearly_growth_df["Yearly Growth"], color='skyblue')
plt.title("Yearly Growth of EV Registrations (Bar Graph)", fontsize=14)
plt.xlabel("Year Range", fontsize=12)
plt.ylabel("Growth in EV Registrations", fontsize=12)
plt.xticks(rotation=45)  # Rotate labels for better visibility
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Line Graph for Yearly Growth with Custom Labels
plt.figure(figsize=(10, 6))
plt.plot(yearly_growth_df["Label"], yearly_growth_df["Yearly Growth"], marker='o', linestyle='-', color='green', linewidth=2)
plt.title("Yearly Growth of EV Registrations (Line Graph)", fontsize=14)
plt.xlabel("Year Range")
plt.ylabel("Growth in EV Registrations")
plt.xticks(rotation=45)  # Rotate labels for better visibility
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.show()

## Scatter Plotting the Correlation Between EV Counts and Charging Stations in SDGE Territories

In [ ]:
sdge_zip_csv = pd.read_csv('data/SDGE_zip.csv')
sdge_service_zip = sdge_zip_csv['ZipCode']
alternative_fuels_data = pd.read_csv('datasets/alternative_fuels_data.csv')
enhanced_columns = [
    'station_name', 'city', 'state', 'zip', 'country', 'access_code',
    'latitude', 'longitude', 'fuel_type_code', 'status_code', 'open_date',
    'ev_connector_types', 'ev_dc_fast_num', 'ev_level1_evse_num',
    'ev_level2_evse_num', 'ev_network', 'ev_network_web', 'ev_other_evse',
    'ev_workplace_charging', 'ev_pricing'
]
alternative_fuels_data = alternative_fuels_data[enhanced_columns]

In [ ]:
charging_station_data = alternative_fuels_data[(alternative_fuels_data['zip'].isin(sdge_service_zip)) & (alternative_fuels_data['fuel_type_code'] == 'ELEC')]
charging_station_data

In [ ]:
# Ensure ZIP codes are strings for consistent merging
SDGE_dmv['Zip_Code'] = SDGE_dmv['Zip_Code'].astype(str)
charging_station_data['zip'] = charging_station_data['zip'].astype(str)

# Filter and count EV charging stations by ZIP code for electric fuel type
charging_stations_by_zip = charging_station_data[charging_station_data['fuel_type_code'] == 'ELEC'].groupby('zip').size().reset_index(name='charger_count')

# Merge EV owners and charging station counts by ZIP code
ev_zip_data = pd.merge(vehicles_by_zip, charging_stations_by_zip, on='zip', how='inner')

# Prepare regression model
X = ev_zip_data['ev_owners']
y = ev_zip_data['charger_count']
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()
predictions = model.predict(X_const)

### Plotly: Interactive Scatter Plot with ZIP Codes in Hover Info ###
fig = go.Figure()

# Add scatter plot with ZIP codes in hover text
fig.add_trace(go.Scatter(
    x=ev_zip_data['ev_owners'], 
    y=ev_zip_data['charger_count'], 
    mode='markers', 
    marker=dict(size=10, opacity=0.7), 
    name='Data points',
    text=ev_zip_data['zip'],  # Add ZIP codes as hover text
    hovertemplate=(
        "<b>ZIP Code:</b> %{text}<br>" +  # Add ZIP code to the hover template
        "<b>EV Owners:</b> %{x}<br>" +  # EV owners count
        "<b>Charging Stations:</b> %{y}<br><extra></extra>"  # Charger count
    )
))

# Add regression line
fig.add_trace(go.Scatter(
    x=ev_zip_data['ev_owners'], 
    y=predictions, 
    mode='lines', 
    line=dict(color='red', width=2), 
    name='Regression line'
))

# Customize layout
fig.update_layout(
    title='Correlation between EV Owners and Charging Stations',
    xaxis_title='Number of EV Owners',
    yaxis_title='Number of EV Charging Stations',
    width=1000,
    height=700
)

fig.show()


## Fitting Poisson Distribution On EV Owner Data 

In [ ]:
# Group data by ZIP code and year
ev_count_by_zip_year = SDGE_dmv.groupby(['Zip_Code', 'Year'])['Vehicles'].sum().reset_index()

# Pivot to create a table for Poisson fitting
pivot_table = ev_count_by_zip_year.pivot(index='Year', columns='Zip_Code', values='Vehicles').fillna(0)

# Prepare to store results
poisson_results = []

# Fit Poisson distribution for each ZIP code
for zip_code in pivot_table.columns:
    ev_counts = pivot_table[zip_code]
    
    # Only fit if there are data points
    if ev_counts.sum() > 0:
        # Fit Poisson using Maximum Likelihood Estimation (MLE)
        model = sm.GLM(ev_counts, np.ones_like(ev_counts), family=sm.families.Poisson())
        result = model.fit()
        
        # Get the lambda (mean of Poisson)
        lambda_estimate = result.mu.mean()
        poisson_results.append({'zip': zip_code, 'lambda': lambda_estimate})

# Convert results to DataFrame
poisson_results_df = pd.DataFrame(poisson_results)

# Save the results to a CSV file
poisson_results_df.to_csv('poisson_results_sdge.csv', index=False)

# Print the first few rows of the results
print(poisson_results_df.head())

## Monte Carlo Simulation

In [ ]:
# Monte Carlo simulation
n_samples = 1000
samples_results = {}

# Extract lambda values for each ZIP code
poisson_params = poisson_results_df.set_index('zip')['lambda'].to_dict()

# Perform sampling
for n in range(n_samples):
    results = []
    for zip_code in poisson_params.keys():
        # Sample from Poisson distribution
        value = poisson.rvs(mu=poisson_params[zip_code], size=1)
        results.append(value[0])
    samples_results[n] = results

# Convert sampling results to DataFrame for analysis
samples_df = pd.DataFrame.from_dict(samples_results, orient='index', columns=poisson_params.keys())

# Save sampled results to a CSV
samples_df.to_csv('poisson_samples.csv', index=False)


In [ ]:
poisson_params

In [ ]:
samples_df.T

In [ ]:
samples_df.sum()

In [ ]:
samples_df.T.sum()

In [ ]:
samples_df.std()

## Visualization For Monte Carlo Simulation of Zip Code 92122

In [ ]:
import matplotlib.pyplot as plt

# Choose a ZIP code to plot the histogram
zip_code_to_plot = '92122'  # Replace with desired ZIP code if known

# Extract samples for the chosen ZIP code
zip_samples = samples_df[zip_code_to_plot]

# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(zip_samples, bins=30, alpha=0.7, color='blue', edgecolor='black')

# Add labels and title
plt.title(f'Histogram of Poisson Samples for ZIP Code {zip_code_to_plot}', fontsize=16)
plt.xlabel('Number of Registered EVs', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Show the plot
plt.tight_layout()
plt.show()